In [1]:
import gc, argparse, sys, os, errno
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#sns.set()
#sns.set_style('whitegrid')
import h5py
from PIL import Image
import os
from tqdm.notebook import tqdm
import scipy
import sklearn
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


## solution:
- groupby: note that in this solution, agg/transform may not be ideal since we should take care of more than one column, so we use apply function, and we could still get the entire row without losing information. Apply allows passing in a function. Examples of using agg and transform: https://zhuanlan.zhihu.com/p/101284491
- pivot table, see: https://zhuanlan.zhihu.com/p/31952948

In [13]:
table = pd.read_excel('pf.xlsx')
table.head()

,交易日期,合约代码,昨结算,今开盘,最高价,最低价,今收盘,今结算,涨跌1,涨跌2,成交量(手),持仓量,增减量,成交额(万元),交割结算价,品种
0,2021-01-04,PF105,6364,6400,6560,6400,6538,6492,174,128,81076,119217,8368,263166.81,,PF
1,2021-01-04,PF106,6354,0,0,0,0,6522,168,168,0,30,0,0.00,,PF
2,2021-01-04,PF107,6434,0,0,0,0,6536,102,102,0,23,0,0.00,,PF
3,2021-01-04,PF108,6436,0,0,0,0,6524,88,88,0,13,0,0.00,,PF
4,2021-01-04,PF109,6404,6478,6580,6442,6562,6518,158,114,446,3196,-3,1453.38,,PF


### solution1, groupby

In [35]:
def get_largest_volume(x, key = '成交量(手)'):
    df = x.sort_values(by = key,ascending=True)
    return df.iloc[-1,:]

table_processed = table.groupby('交易日期  ',as_index=False).apply(get_largest_volume)

In [36]:
table_processed.head(10)

,交易日期,合约代码,昨结算,今开盘,最高价,最低价,今收盘,今结算,涨跌1,涨跌2,成交量(手),持仓量,增减量,成交额(万元),交割结算价,品种
0,2021-01-04,PF105,6364,6400,6560,6400,6538,6492,174,128,81076,119217,8368,263166.81,,PF
1,2021-01-05,PF105,6492,6522,6696,6514,6594,6606,102,114,242893,130278,11061,802116.93,,PF
2,2021-01-06,PF105,6606,6610,6746,6580,6714,6674,108,68,199265,131341,1063,664926.72,,PF
3,2021-01-07,PF105,6674,6702,6866,6666,6848,6770,174,96,166757,144270,12929,564380.05,,PF
4,2021-01-08,PF105,6770,6844,6910,6724,6804,6824,34,54,177939,134523,-9747,607147.50,,PF
5,2021-01-11,PF105,6824,6814,6818,6548,6572,6666,-252,-158,231048,115681,-18842,770132.35,,PF
6,2021-01-12,PF105,6666,6576,6662,6512,6646,6588,-20,-78,115900,113051,-2630,381723.38,,PF
7,2021-01-13,PF105,6588,6660,6772,6626,6726,6704,138,116,152543,113824,773,511305.67,,PF
8,2021-01-14,PF105,6704,6712,6738,6592,6596,6658,-108,-46,140959,106915,-6909,469282.01,,PF
9,2021-01-15,PF105,6658,6582,6682,6546,6576,6612,-82,-46,133221,112056,5141,440418.22,,PF


### solution2 pivot table
use index and values could form the pivot table you need!

In [51]:
table.head()

,交易日期,合约代码,昨结算,今开盘,最高价,最低价,今收盘,今结算,涨跌1,涨跌2,成交量(手),持仓量,增减量,成交额(万元),交割结算价,品种
0,2021-01-04,PF105,6364,6400,6560,6400,6538,6492,174,128,81076,119217,8368,263166.81,,PF
1,2021-01-04,PF106,6354,0,0,0,0,6522,168,168,0,30,0,0.00,,PF
2,2021-01-04,PF107,6434,0,0,0,0,6536,102,102,0,23,0,0.00,,PF
3,2021-01-04,PF108,6436,0,0,0,0,6524,88,88,0,13,0,0.00,,PF
4,2021-01-04,PF109,6404,6478,6580,6442,6562,6518,158,114,446,3196,-3,1453.38,,PF


In [71]:
pivot_table = pd.pivot_table(table,index=['交易日期  ','合约代码'],values=['成交量(手)'])
pivot_table

成交量(手)
交易日期       合约代码            
2021-01-04 PF105      81076
           PF106          0
           PF107          0
           PF108          0
           PF109        446
...                     ...
2021-05-19 PF112          3
           PF201        309
           PF202          3
           PF203          5
           PF204         12

[915 rows x 1 columns]

In [80]:
pivot_table = pd.pivot_table(table,index=['交易日期  '],values=['成交量(手)','合约代码'],aggfunc=get_largest_volume)
pivot_table #if you want to show more columns, add column names to values list

,合约代码,成交量(手)
交易日期,,
2021-01-04,PF105,81076
2021-01-05,PF105,242893
2021-01-06,PF105,199265
2021-01-07,PF105,166757
2021-01-08,PF105,177939
...,...,...
2021-05-13,PF109,218177
2021-05-14,PF109,251882
2021-05-17,PF109,195417


### solution: bruteforce
- select subtable
- sort by key, select
- concat

In [66]:
unique_dates = np.unique(table['交易日期  '])
table_processed = pd.DataFrame(columns=table.columns)

for date in tqdm(unique_dates):
    display(table[table['交易日期  ']==date])  #this could extract each day's subtable

,交易日期,合约代码,昨结算,今开盘,最高价,最低价,今收盘,今结算,涨跌1,涨跌2,成交量(手),持仓量,增减量,成交额(万元),交割结算价,品种
0,2021-01-04,PF105,6364,6400,6560,6400,6538,6492,174,128,81076,119217,8368,263166.81,,PF
1,2021-01-04,PF106,6354,0,0,0,0,6522,168,168,0,30,0,0.00,,PF
2,2021-01-04,PF107,6434,0,0,0,0,6536,102,102,0,23,0,0.00,,PF
3,2021-01-04,PF108,6436,0,0,0,0,6524,88,88,0,13,0,0.00,,PF
4,2021-01-04,PF109,6404,6478,6580,6442,6562,6518,158,114,446,3196,-3,1453.38,,PF
5,2021-01-04,PF110,6438,0,0,0,0,6474,36,36,0,10,0,0.00,,PF
6,2021-01-04,PF111,6418,0,0,0,0,6494,76,76,0,3,0,0.00,,PF
7,2021-01-04,PF112,6368,0,0,0,0,6452,84,84,0,0,0,0.00,,PF


,交易日期,合约代码,昨结算,今开盘,最高价,最低价,今收盘,今结算,涨跌1,涨跌2,成交量(手),持仓量,增减量,成交额(万元),交割结算价,品种
8,2021-01-05,PF105,6492,6522,6696,6514,6594,6606,102,114,242893,130278,11061,802116.93,,PF
9,2021-01-05,PF106,6522,6620,6620,6620,6620,6620,98,98,1,31,1,3.31,,PF
10,2021-01-05,PF107,6536,0,0,0,0,6596,60,60,0,23,0,0.00,,PF
11,2021-01-05,PF108,6524,0,0,0,0,6600,76,76,0,13,0,0.00,,PF
12,2021-01-05,PF109,6518,6562,6714,6560,6624,6628,106,110,1414,3099,-97,4685.91,,PF
13,2021-01-05,PF110,6474,6570,6570,6570,6570,6570,96,96,1,9,-1,3.29,,PF
14,2021-01-05,PF111,6494,0,0,0,0,6580,86,86,0,3,0,0.00,,PF
15,2021-01-05,PF112,6452,0,0,0,0,6540,88,88,0,0,0,0.00,,PF


,交易日期,合约代码,昨结算,今开盘,最高价,最低价,今收盘,今结算,涨跌1,涨跌2,成交量(手),持仓量,增减量,成交额(万元),交割结算价,品种
16,2021-01-06,PF105,6606,6610,6746,6580,6714,6674,108,68,199265,131341,1063,664926.72,,PF
17,2021-01-06,PF106,6620,6622,6702,6524,6702,6644,82,24,16,30,-1,53.15,,PF
18,2021-01-06,PF107,6596,6634,6706,6530,6686,6668,90,72,23,25,2,76.68,,PF
19,2021-01-06,PF108,6600,6646,6726,6646,6706,6696,106,96,5,12,-1,16.74,,PF
20,2021-01-06,PF109,6628,6664,6778,6616,6740,6708,112,80,615,3167,68,2062.51,,PF
21,2021-01-06,PF110,6570,6690,6690,6680,6680,6686,110,116,5,8,-1,16.72,,PF
22,2021-01-06,PF111,6580,6700,6738,6676,6730,6730,150,150,158,154,151,531.57,,PF
23,2021-01-06,PF112,6540,0,0,0,0,6666,126,126,0,0,0,0.00,,PF


,交易日期,合约代码,昨结算,今开盘,最高价,最低价,今收盘,今结算,涨跌1,涨跌2,成交量(手),持仓量,增减量,成交额(万元),交割结算价,品种
24,2021-01-07,PF105,6674,6702,6866,6666,6848,6770,174,96,166757,144270,12929,564380.05,,PF
25,2021-01-07,PF106,6644,6794,6794,6794,6794,6794,150,150,1,30,0,3.40,,PF
26,2021-01-07,PF107,6668,6766,6766,6766,6766,6766,98,98,1,25,0,3.38,,PF
27,2021-01-07,PF108,6696,0,0,0,0,6856,160,160,0,12,0,0.00,,PF
28,2021-01-07,PF109,6708,6730,6882,6702,6866,6788,158,80,1470,3493,326,4989.02,,PF
29,2021-01-07,PF110,6686,0,0,0,0,6832,146,146,0,8,0,0.00,,PF
30,2021-01-07,PF111,6730,6764,6822,6764,6822,6794,92,64,2,155,1,6.79,,PF
31,2021-01-07,PF112,6666,0,0,0,0,6800,134,134,0,0,0,0.00,,PF


,交易日期,合约代码,昨结算,今开盘,最高价,最低价,今收盘,今结算,涨跌1,涨跌2,成交量(手),持仓量,增减量,成交额(万元),交割结算价,品种
32,2021-01-08,PF105,6770,6844,6910,6724,6804,6824,34,54,177939,134523,-9747,607147.50,,PF
33,2021-01-08,PF106,6794,6860,6874,6802,6822,6852,28,58,35,29,-1,119.88,,PF
34,2021-01-08,PF107,6766,6882,6882,6882,6882,6882,116,116,1,26,1,3.44,,PF
35,2021-01-08,PF108,6856,0,0,0,0,6850,-6,-6,0,12,0,0.00,,PF
36,2021-01-08,PF109,6788,6854,6922,6752,6838,6874,50,86,1283,3853,360,4409.57,,PF
37,2021-01-08,PF110,6832,0,0,0,0,6832,0,0,0,8,0,0.00,,PF
38,2021-01-08,PF111,6794,6784,6914,6784,6914,6892,120,98,605,754,599,2084.91,,PF
39,2021-01-08,PF112,6800,0,0,0,0,6800,0,0,0,0,0,0.00,,PF


...

,交易日期,合约代码,昨结算,今开盘,最高价,最低价,今收盘,今结算,涨跌1,涨跌2,成交量(手),持仓量,增减量,成交额(万元),交割结算价,品种
855,2021-05-13,PF105,6782,6820,6900,6486,6486,6536,-296,-246,918,5634,-731,3107.25,"6,794.00",PF
856,2021-05-13,PF106,7254,7324,7324,7004,7004,7156,-250,-98,4,337,-1,14.31,,PF
857,2021-05-13,PF107,7242,7338,7402,7024,7064,7244,-178,2,206867,78647,-4441,749152.34,,PF
858,2021-05-13,PF108,7328,7354,7354,7344,7344,7350,16,22,2,16,0,7.35,,PF
859,2021-05-13,PF109,7318,7400,7476,7102,7152,7310,-166,-8,218177,130976,-9475,797384.47,,PF
860,2021-05-13,PF110,7372,7492,7518,7246,7246,7382,-126,10,7,2206,5,25.84,,PF
861,2021-05-13,PF111,7420,7562,7568,7298,7318,7420,-102,0,20,2381,14,74.20,,PF
862,2021-05-13,PF112,7454,7478,7478,7394,7394,7434,-60,-20,8,1976,3,29.73,,PF
863,2021-05-13,PF201,7480,7512,7602,7300,7326,7422,-154,-58,227,1471,70,842.40,,PF
864,2021-05-13,PF202,7506,0,0,0,0,7462,-44,-44,0,668,0,0.00,,PF


,交易日期,合约代码,昨结算,今开盘,最高价,最低价,今收盘,今结算,涨跌1,涨跌2,成交量(手),持仓量,增减量,成交额(万元),交割结算价,品种
867,2021-05-14,PF105,6536,6504,6560,6332,6332,6492,-204,-44,477,5163,-471,1558.73,"6,778.00",PF
868,2021-05-14,PF106,7156,6930,6930,6796,6796,6822,-360,-334,14,328,-9,47.76,,PF
869,2021-05-14,PF107,7244,7050,7106,6808,6832,6948,-412,-296,118519,59737,-18910,411759.96,,PF
870,2021-05-14,PF108,7350,0,0,0,0,7050,-300,-300,0,16,0,0.00,,PF
871,2021-05-14,PF109,7310,7148,7184,6880,6908,7002,-402,-308,251882,131049,73,881665.41,,PF
872,2021-05-14,PF110,7382,7102,7102,6976,7018,7036,-364,-346,14,2216,10,49.25,,PF
873,2021-05-14,PF111,7420,7198,7198,7026,7052,7100,-368,-320,27,2403,22,95.83,,PF
874,2021-05-14,PF112,7434,0,0,0,0,7142,-292,-292,0,1976,0,0.00,,PF
875,2021-05-14,PF201,7422,7268,7336,7076,7124,7194,-298,-228,500,1657,186,1798.43,,PF
876,2021-05-14,PF202,7462,7320,7320,7092,7188,7174,-274,-288,413,1069,401,1481.14,,PF


,交易日期,合约代码,昨结算,今开盘,最高价,最低价,今收盘,今结算,涨跌1,涨跌2,成交量(手),持仓量,增减量,成交额(万元),交割结算价,品种
879,2021-05-17,PF105,6492,6402,6572,6380,6520,6450,28,-42,131,5093,-70,436.20,"6,756.00",PF
880,2021-05-17,PF106,6822,6796,6796,6796,6796,6796,-26,-26,1,327,-1,3.40,,PF
881,2021-05-17,PF107,6948,6898,6980,6858,6900,6912,-48,-36,48670,59160,-577,168186.51,,PF
882,2021-05-17,PF108,7050,6922,6974,6896,6904,6934,-146,-116,7,13,-3,24.27,,PF
883,2021-05-17,PF109,7002,6968,7060,6930,6976,6988,-26,-14,195417,139919,8870,682817.03,,PF
884,2021-05-17,PF110,7036,7108,7108,7054,7084,7070,48,34,8,2220,4,28.28,,PF
885,2021-05-17,PF111,7100,7102,7144,7102,7142,7116,42,16,14,2415,12,49.81,,PF
886,2021-05-17,PF112,7142,7170,7172,7170,7172,7172,30,30,3,1978,2,10.76,,PF
887,2021-05-17,PF201,7194,7194,7232,7160,7204,7194,10,0,349,1818,161,1255.13,,PF
888,2021-05-17,PF202,7174,7210,7220,7210,7220,7212,46,38,11,1060,-9,39.66,,PF


,交易日期,合约代码,昨结算,今开盘,最高价,最低价,今收盘,今结算,涨跌1,涨跌2,成交量(手),持仓量,增减量,成交额(万元),交割结算价,品种
891,2021-05-18,PF105,6450,6382,6696,6382,6696,6582,246,132,119,5034,-59,398.58,"6,736.00",PF
892,2021-05-18,PF106,6796,0,0,0,0,6916,120,120,0,327,0,0.00,,PF
893,2021-05-18,PF107,6912,6898,7000,6860,6964,6934,52,22,35187,57863,-1297,121997.04,,PF
894,2021-05-18,PF108,6934,0,0,0,0,6948,14,14,0,13,0,0.00,,PF
895,2021-05-18,PF109,6988,6976,7082,6934,7050,7020,62,32,160992,143160,3241,565000.52,,PF
896,2021-05-18,PF110,7070,7162,7182,7154,7182,7166,112,96,23,2221,1,82.40,,PF
897,2021-05-18,PF111,7116,7240,7240,7240,7240,7240,124,124,1,2415,0,3.62,,PF
898,2021-05-18,PF112,7172,0,0,0,0,7200,28,28,0,1978,0,0.00,,PF
899,2021-05-18,PF201,7194,7102,7276,7102,7254,7238,60,44,151,1812,-6,546.40,,PF
900,2021-05-18,PF202,7212,7212,7212,7212,7212,7212,0,0,1,1060,0,3.61,,PF


,交易日期,合约代码,昨结算,今开盘,最高价,最低价,今收盘,今结算,涨跌1,涨跌2,成交量(手),持仓量,增减量,成交额(万元),交割结算价,品种
903,2021-05-19,PF105,6582,6620,6620,6542,6542,6558,-40,-24,162,0,-5034,531.18,"6,712.00",PF
904,2021-05-19,PF106,6916,6922,6922,6784,6784,6802,-132,-114,28,301,-26,95.23,,PF
905,2021-05-19,PF107,6934,6960,6990,6826,6876,6898,-58,-36,27359,56599,-1264,94347.15,,PF
906,2021-05-19,PF108,6948,0,0,0,0,6948,0,0,0,13,0,0.00,,PF
907,2021-05-19,PF109,7020,7056,7074,6904,6962,6978,-58,-42,168846,138566,-4594,589067.76,,PF
908,2021-05-19,PF110,7166,0,0,0,0,7096,-70,-70,0,2221,0,0.00,,PF
909,2021-05-19,PF111,7240,7118,7146,7118,7146,7132,-94,-108,2,2417,2,7.13,,PF
910,2021-05-19,PF112,7200,7102,7114,7102,7114,7106,-86,-94,3,1980,2,10.66,,PF
911,2021-05-19,PF201,7238,7222,7268,7140,7208,7184,-30,-54,309,1759,-53,1109.96,,PF
912,2021-05-19,PF202,7212,7192,7202,7192,7202,7200,-10,-12,3,1061,1,10.80,,PF


In [67]:
for date in tqdm(unique_dates):
    tmptable = table[table['交易日期  ']==date].sort_values(by = '成交量(手)',ascending=True).iloc[-1:] #extract each day's table, sort by '成交量(手)', ascending order, so we select last row!
    table_processed = pd.concat((table_processed,tmptable))

In [68]:
table_processed

,交易日期,合约代码,昨结算,今开盘,最高价,最低价,今收盘,今结算,涨跌1,涨跌2,成交量(手),持仓量,增减量,成交额(万元),交割结算价,品种
0,2021-01-04,PF105,6364,6400,6560,6400,6538,6492,174,128,81076,119217,8368,263166.81,,PF
8,2021-01-05,PF105,6492,6522,6696,6514,6594,6606,102,114,242893,130278,11061,802116.93,,PF
16,2021-01-06,PF105,6606,6610,6746,6580,6714,6674,108,68,199265,131341,1063,664926.72,,PF
24,2021-01-07,PF105,6674,6702,6866,6666,6848,6770,174,96,166757,144270,12929,564380.05,,PF
32,2021-01-08,PF105,6770,6844,6910,6724,6804,6824,34,54,177939,134523,-9747,607147.50,,PF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,2021-05-13,PF109,7318,7400,7476,7102,7152,7310,-166,-8,218177,130976,-9475,797384.47,,PF
871,2021-05-14,PF109,7310,7148,7184,6880,6908,7002,-402,-308,251882,131049,73,881665.41,,PF
883,2021-05-17,PF109,7002,6968,7060,6930,6976,6988,-26,-14,195417,139919,8870,682817.03,,PF
895,2021-05-18,PF109,6988,6976,7082,6934,7050,7020,62,32,160992,143160,3241,565000.52,,PF
